In [6]:
fileList=[]

import os
indir = '/Users/brisbin/desktop/blastXML'
for root, dirs, filenames in os.walk(indir):
    for f in filenames:
        fileList.append(f)
        
fileList.pop(0)


'.DS_Store'

In [4]:
geneIDlist=[]
giNUMlist=[]

import pdb
for i in fileList:
    x='/Users/brisbin/desktop/blastXML/' + i
    blast = NCBIXML.parse(open(x,'rU'))
    record = blast.next()
    while record:
        if record.alignments:
        #print gene id
             geneIDlist.append(record.query)
        # to print the "best" matches title
             giNUMlist.append(record.alignments[0].title)
        try:
            record = blast.next()
        except:
            break

In [5]:
len(geneIDlist)

33261

In [10]:
newGENElist=[]
for i in range(0,len(geneIDlist)):
    newGENE= (geneIDlist[i].split(" ")[0])
    newGENElist.append(newGENE)

len(newGENElist)  


33261

In [59]:
## Species
SPlist=[]
for i in range(0,len(giNUMlist)):
    newSP= (giNUMlist[i].split("|")[6])
    SPlist.append(newSP)

## Species Continued
SPlist2=[]
for i in range(0,len(giNUMlist)):
    newSP= (SPlist[i].split("[")[-1])
    SPlist2.append(newSP)

#Species list finished
 
SPlist3=[]
for i in range(0,len(giNUMlist)):
    newSP= (SPlist2[i].split("]")[0])
    SPlist3.append(newSP)
    
len(SPlist3)

33261

In [60]:
if 'Rous sarcoma virus' in SPlist3:
    print 'yes'
SPlist3.index('Rous sarcoma virus')

yes


9154

In [44]:
SPlist3.remove('Rous sarcoma virus')

In [45]:
len(SPlist3)

33260

In [61]:
del newGENElist[9154]

In [62]:
# create a dictionary with the key as the gene id and the value as the species name
GeneSpeciesDict={}

for i in range(0,len(newGENElist)):
    newkey = newGENElist[i]
    GeneSpeciesDict[newkey]=SPlist3[i]


In [63]:
import itertools

def glance(d):
    return dict(itertools.islice(d.iteritems(), 3))

glance(GeneSpeciesDict)

{u'TRINITY_DN117013_c0_g1_i1': u'Scleropages formosus',
 u'TRINITY_DN203990_c0_g1_i1': u'Austrofundulus limnaeus',
 u'TRINITY_DN208795_c0_g1_i1': u'Neolamprologus brichardi'}

In [67]:
from Bio import Entrez
Entrez.email="margaret.marsbrisbin@oist.jp"

taxID = {}

for i in SPlist3:
    if i in taxID:
        continue
    handle = Entrez.esearch(db="taxonomy", term= i)
    record = Entrez.read(handle)
    #if not record["IdList"]:
    #    pdb.set_trace()
    if record["IdList"]:
        taxID[i] = record["IdList"][0]
#    taxIDlist.append(record["IdList"])
    

In [68]:
glance(taxID)

{u'Balaenoptera acutorostrata scammoni': '310752',
 u'Pseudomonas putida': '303',
 u'Shewanella frigidimarina': '56812'}

In [69]:
from ete2 import NCBITaxa
ncbi = NCBITaxa()

In [71]:
lineageDict={}

for i in taxID:
    lineage = ncbi.get_lineage(taxID[i])
    names = ncbi.get_taxid_translator(lineage)
    key = taxID[i]
    lineageDict[key]= [names[taxid] for taxid in lineage]

In [81]:
#glance(lineageDict)

In [82]:
# make a dictionary for Fish or Not Fish by tax id
# 0 = Not Fish, 1 = Fish
fishDict = {} 

keylist=lineageDict.keys()

for i in keylist:
    key = i
    value = lineageDict[key]
    if 'Teleostei' in value:
        binary = 1
    else:
        binary = 0
    fishDict[key]= binary

In [83]:
glance(fishDict)

{'1234592': 0, '59894': 0, '696485': 0}

In [84]:
#Get rid of geneIDs that do not have taxonomy available (no species name in title)
import itertools

def glance(d):
    return dict(itertools.islice(d.iteritems(), 3))


keylist = GeneSpeciesDict.keys()
for i in keylist:
    if i in GeneSpeciesDict:
        if len(GeneSpeciesDict[i])> 50:
            del GeneSpeciesDict[i] 

switchTaxID = {y:x for x,y in taxID.iteritems()} 

In [85]:
glance(switchTaxID)

{'1234592': u'Vibrio sp. 712i1',
 '59894': u'Ficedula albicollis',
 '696485': u'Vibrio owensii'}

In [86]:
keylist2 = fishDict.keys()
fishDict2={}

for i in keylist2:
    key = switchTaxID[i]
    value = fishDict[i]
    fishDict2[key]=value

glance(fishDict2)

{u'Balaenoptera acutorostrata scammoni': 0,
 u'Pennahia argentata': 1,
 u'Tasmanian devil retrovirus': 0}

In [88]:
keylist3=GeneSpeciesDict.keys()
lastDict={}
for i in keylist3:
    if GeneSpeciesDict[i] in fishDict2:
        newval= fishDict2[(GeneSpeciesDict[i])]
        lastDict[i]= newval

glance(lastDict)

{u'TRINITY_DN117013_c0_g1_i1': 1,
 u'TRINITY_DN203990_c0_g1_i1': 1,
 u'TRINITY_DN208795_c0_g1_i1': 1}

In [90]:
with open('FishNotFish.csv', 'w') as f:
    [f.write('{0},{1}\n'.format(key, value)) for key, value in lastDict.items()]